In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('test.csv')
df_survived=pd.read_csv('gender_submission.csv')

In [3]:
df=pd.concat([df,df_survived['Survived']],axis=1)

In [4]:
df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,1
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,1


Checking and dropping NaN items


In [5]:
df.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
Survived         0
dtype: int64

In [6]:
df['Fare'].fillna(df['Fare'].mean(),inplace=True) #Replacing with mean values
df['Age'].fillna(df['Age'].mean(),inplace=True) #Replacing with mean values
df.drop('Cabin',axis=1,inplace=True)

<h1>Detailed_Details Function</h1>
<h2> Takes 4 inputs </h2>
<ul>
<li> df = dataframe </li>
<li> target= Targeted Feature </li>
<li> outcome= Targeted Outcome </li>
<li> max_unique_value_count = (Set)Maximum Unique Value Count in between Target and Outcome.Best to set Max_Unique_Count Finite (i.e Less than 25).<br> That is "df[target].nunique()<=25" or "df[outcome].nunique()<=25" <br> Now Choose Max Value Between them.</li>
</ul>
<h3> Remove all NaN values before Hand. </h3>
<h3> If you have dropped rows in Dataframe. ReIndex Your DataFrame (DataFrame.reindex()). </h3>
<h3> This function doesnot evaluate for strings or objects having large unique_value_counts. </h3>

In [16]:


#if you have dropped rows you have to reindex to use this function.
def Detailed_Details(df,feature,outcome,max_unique_value_count):
    from IPython.display import HTML, display
    html=f"<h3><center>Comparing {feature} With {outcome}</center></h3>"
    
    
    #for checking type
    type={'int','int16','int32','int64','float','float16','float32','float64'}
    
    uniq_cases=max_unique_value_count
    total_data=len(df)
    
    #defining feature
    df_feature=df[feature]
    check_type=df_feature.dtypes
    
    
    #defining outcome
    df_outcome=df[outcome]
    check_type_outcome=df_outcome.dtypes
    check_type_feature=df_feature.dtypes
    #designing vars
    design=0
    design_feature=0
    if(str(check_type_outcome) in type or str(check_type_feature) in type):   # Checks if feature is of int or float data type
        
        if(df_feature.nunique()<=uniq_cases): #Checks if feature has less unique values
            
            uniq_sorted_ft=sorted(df_feature.unique()) #Sorts features unique values
            
            #Unique occurence of features
            feature_count=np.array(list(df[feature].value_counts().sort_index()))
            
            #Percentage of each occurence of Feature
            feature_percent=[]
            for i in range(0,len(feature_count)):
                feature_percent.append(round(((feature_count[i]/total_data)*100),2))
            feature_percent=np.array(feature_percent)
            
            #UNIQUE NUMERICAL FEATURES AND UNIQUE NUMERICAL OUTCOMES
            if(df_outcome.nunique()<=uniq_cases):
                
                uniq_sorted_ot=sorted(df_outcome.unique()) #list to put unique values for outcome
                
                #Each Unique Occurence of features and outcomes simultanously
                outcome_count = np.array(list(df.groupby([outcome,feature]).size().unstack(fill_value=0).stack()))
                
                #Percentage of Occurence of features with outcome
                
                #partioning feature with respect to outcome
                partition=int(((len(outcome_count)/len(feature_count))or(len(feature_count)/len(outcome_count))))
                #so that percent will be float
                outcome_count_float=[]
                for i in range(0,len(outcome_count)):
                    outcome_count_float.append(float(outcome_count[i]))
                #Calculating outcome percentage
                outcome_percent=[]
                for i in range(0, len(outcome_count_float), len(feature_count)):
                    outcome_percent.append(outcome_count_float[i:i+len(feature_count)])
                
                for i in range(0,len(feature_count)):
                    for j in range(0,partition):
                        outcome_percent[j][i]=((outcome_percent[j][i]/feature_count[i])*100)
                        outcome_percent[j][i]=round(outcome_percent[j][i],2)
                outcome_percent=np.array(outcome_percent)
                
                #indexes of outcome data
                indexes=[]
                for uniq in uniq_sorted_ot:
                    indexes.append(uniq_sorted_ot.index(uniq))
                
                #Display
                html+=f"Total Features {len(df)} <br>"
                html+="<center><table cellpadding=15>"
                    
                if design==0:
                    html+="<tr>"
                    html+=f"<td><center>{feature}</center> </td> <td>Total No. ({feature})</td> <td> Percentage ({feature}) </td>"
                    for i in indexes:
                        html+=f"<td>Total Outcome({uniq_sorted_ot[i]})</td> <td>Percentage Outcome({uniq_sorted_ot[i]})</td> "
                    html+="</tr>"
                    design=1
                for i in range (0,len(feature_count)):
                    html+="<tr>"
                    for j in range(0,partition):
                        if design_feature==0:
                            html+=f"<td></center>{uniq_sorted_ft[i]}</center></td><td>{feature_count[i]}</td><td>{feature_percent[i]}%</td>"
                        if design_feature==0:
                            html+=f"<td>{outcome_count[i]}</td><td>{outcome_percent[j][i]}%</td>"
                        else:
                            html+=f"<td>{outcome_count[i+len(feature_count)*j]}</td><td>{outcome_percent[j][i]}%</td>"
                        design_feature=1
                    html+=f"</tr>"
                    design_feature=0
                html+="<table></center>"
                display(HTML(html))
                #UNIQUE NUMERICAL FEATURES AND MULTIPLE NUMERICAL OUTCOME
            elif(df_outcome.nunique()>uniq_cases and str(check_type_outcome) in type):
                mean_value=round(df_outcome.mean(),2)
                
                #Dict for Total and percent
                total={}
                percent={}
                #Dict for total above and below mean and thier percentage
                total_outcome_abm={}
                total_outcome_blm={}
                percent_outcome_abm={}
                percent_outcome_blm={}
                #initializing dictionaries with 0
                for i in uniq_sorted_ft:
                    total_outcome_abm[i]=0
                    total_outcome_blm[i]=0
                    total[i]=0
                    percent[i]=0
                    percent_outcome_abm[i]=0
                    percent_outcome_blm[i]=0
                #Count for total above and below mean
                for uniq in uniq_sorted_ft:
                    for i in range(0,len(df)):
                        if(df_outcome[i]>=mean_value):
                            if(df[feature][i]==uniq):
                                total_outcome_abm[uniq]+=1
                        else:
                            if(df[feature][i]==uniq):
                                total_outcome_blm[uniq]+=1
                #Count total in each category
                for uniq in uniq_sorted_ft:
                    total[uniq]=total_outcome_abm[uniq]+total_outcome_blm[uniq]
                #Count percent in each category
                for uniq in uniq_sorted_ft:
                    percent[uniq]=round(total[uniq]/total_data*100,2)
                    percent_outcome_abm[uniq]=round(total_outcome_abm[uniq]/total[uniq]*100,2)
                    percent_outcome_blm[uniq]=round(total_outcome_blm[uniq]/total[uniq]*100,2)
                                             
                #Display
                if design==0:
                    html+=f"Total Features {len(df)}<br>"
                    html+=f"Mean of {outcome} = {mean_value}<br>"
                    html+="<center><table cellpadding=15>"
                    html+="<tr>"
                    html+=f"<td><center>{feature}</center> </td> <td>Total No. <br>({feature})</td> <td> Percentage <br>({feature}) </td>"
                    html+=f"<td>Total No.<br>(Greater Than Mean)</td> <td>Percentage<br>(Greater Than Mean)</td><td>Total No.<br>(Less Than Mean)</td> <td>Percentage <br>(Less Than Mean)</td> "
                    html+="</tr>"
                    design=1
                
                design_feature=0
                for uniq in uniq_sorted_ft:
                    
                    html+="<tr>"    
                    html+=f"<td>{uniq}</td> <td>{total[uniq]}</td><td>{percent[uniq]} %</td><td>{total_outcome_abm[uniq]}</td><td>{percent_outcome_abm[uniq]} %</td><td>{total_outcome_blm[uniq]}</td><td>{percent_outcome_blm[uniq]} %</td>"           
                    html+=f"</tr>"
                html+="</table></center>"
                
                display(HTML(html))
        elif(df_feature.nunique()>uniq_cases and str(check_type_feature)  in type):
                
                mean_value=round(df_feature.mean(),2)
                #Dict for Total and percent
                total={}
                percent={}
                total_abm=0
                total_blm=0
                #Dict for total above and below mean and thier percentage
                total_feature_abm={}
                total_feature_blm={}
                percent_feature_abm={}
                percent_feature_blm={}
                #initializing dictionaries with 0
                uniq_sorted_ot=sorted(df_outcome.unique())
                for i in uniq_sorted_ot:
                    total_feature_abm[i]=0
                    total_feature_blm[i]=0
                    total[i]=0
                    percent[i]=0
                    percent_feature_abm[i]=0
                    percent_feature_blm[i]=0
                #Count below mean and above mean
                for i in range(0,len(df)):
                    if(df_feature[i]>=mean_value):
                        total_abm+=1
                    else:
                        total_blm+=1
                #Count for total above and below mean
                for uniq in uniq_sorted_ot:
                    for i in range(0,len(df)):
                        if(df_feature[i]>=mean_value):
                            if(df[outcome][i]==uniq):
                                total_feature_abm[uniq]+=1
                        else:
                            if(df[outcome][i]==uniq):
                                total_feature_blm[uniq]+=1
                #Count total in each category
                for uniq in uniq_sorted_ot:
                    total[uniq]=total_feature_abm[uniq]+total_feature_blm[uniq]
                #Count percent in each category
                for uniq in uniq_sorted_ot:
                    percent[uniq]=round(total[uniq]/total_data*100,2)
                    percent_feature_abm[uniq]=round(total_feature_abm[uniq]/total[uniq]*100,2)
                    percent_feature_blm[uniq]=round(total_feature_blm[uniq]/total[uniq]*100,2)
                
                #MULTIPLE NUMERICAL FEATURES AND UNIQUE NUMERICAL OUTCOMES 
                if(df_outcome.nunique()<=uniq_cases):
                    uniq_sorted_ot=sorted(df_outcome.unique()) #Sorts features unique values
                   
                    #Unique occurence of features
                    outcome_count=np.array(list(df[outcome].value_counts().sort_index()))
            
                    #Percentage of each occurence of Feature
                    outcome_percent=[]
                    for i in range(0,len(outcome_count)):
                        outcome_percent.append(round(((outcome_count[i]/total_data)*100),2))
                    outcome_percent=np.array(outcome_percent)
            
                    #Checks if outcome has less unique values
                                             
                    #Display
                    if design==0:
                        html+=f"Total Features {len(df)}<br>"
                        html+=f"Mean of {feature} = {mean_value}<br>"
                        html+="<center><table cellpadding=15>"
                        html+="<tr>"
                        html+=f"<td><center>{feature}</center> </td> <td>Total No. <br>({feature})</td> <td> Percentage <br>({feature}) </td>"
                        for uniq in uniq_sorted_ot:
                            html+=f"<td>Total Outcome<br>({uniq})</td><td>Percentage Outcome<br>({uniq})"
                        html+="</td>"
                        html+="</tr>"
                        design=1
                
                    design_feature=0
                    html+=f"<tr><td>Greater Than Mean</td><td>{total_abm}</td><td>{round((total_abm/total_data)*100,2)}</td>"
                    for uniq in uniq_sorted_ot:
                        html+=f"<td>{total_feature_abm[uniq]}</td><td>{percent_feature_abm[uniq]} %</td>"
                    html+=f"<tr><td>Less Than Mean</td><td>{total_blm}</td><td>{round((total_blm/total_data)*100,2)}</td>"
                    for uniq in uniq_sorted_ot:
                        html+=f"<td>{total_feature_blm[uniq]}</td><td>{percent_feature_blm[uniq]} %</td>"  
                    html+="</table></center>"
                
                    display(HTML(html))
                #MULTIPLE NUMERICAL FEATURES AND MULTIPLE NUMERICAL OUTCOMES
                elif(df_outcome.nunique()>uniq_cases and str(check_type_outcome)  in type):
                    mean_value_outcome=round(df_outcome.mean(),2)
                    
                    total_out_abm_fet_abm=0
                    total_out_abm_fet_blm=0
                    total_out_blm_fet_abm=0
                    total_out_blm_fet_blm=0
                        
                    #Count for total above and below mean
                    for i in range(0,len(df)):
                        if(df_outcome[i]>=mean_value_outcome):
                            if(df[feature][i]>=mean_value):
                                total_out_abm_fet_abm+=1
                            else:
                                total_out_abm_fet_blm+=1
                        else:
                            if(df[feature][i]>=mean_value):
                                total_out_blm_fet_abm+=1
                            else:
                                total_out_blm_fet_blm+=1
                    per_out_abm_fet_abm=round((total_out_abm_fet_abm/total_abm*100),2)
                    per_out_abm_fet_blm=round((total_out_abm_fet_blm/total_blm*100),2)
                    per_out_blm_fet_abm=round((100-per_out_abm_fet_abm),2)
                    per_out_blm_fet_blm=round((100-per_out_abm_fet_blm),2)      
                    
                    
                    #Display
                    if design==0:
                        html+=f"Total Features {len(df)}<br>"
                        html+=f"Mean of {feature} = {mean_value}<br>"
                        html+=f"Mean of {outcome} ={mean_value_outcome}<br>"
                        html+="<center><table cellpadding=15>"
                        html+="<tr>"
                        html+=f"<td><center>{feature}</center> </td> <td>Total No. <br>({feature})</td> <td> Percentage <br>({feature}) </td>"
                        html+=f"<td> Total Outcome <br>Greater Than Mean<br>({mean_value_outcome})</td><td> Percentage Outcome <br>Greater Than Mean<br>({mean_value_outcome})</td>"
                        html+=f"<td> Total Outcome <br>Less Than Mean<br>({mean_value_outcome})</td><td> Percentage Outcome <br>Less Than Mean<br>({mean_value_outcome})</td>"
                        html+="</td>"
                        html+="</tr>"
                        design=1
                
                    design_feature=0
                    html+=f"<tr><td>Greater Than Mean<br>({mean_value})</td><td>{total_abm}</td><td>{round((total_abm/total_data)*100,2)} %</td><td>{total_out_abm_fet_abm}</td><td>{per_out_abm_fet_abm} %</td><td>{total_out_blm_fet_abm}</td><td>{per_out_blm_fet_abm} %</td>"
                    
                    html+=f"<tr><td>Less Than Mean<br>({mean_value})</td><td>{total_blm}</td><td>{round((total_blm/total_data)*100,2)} %</td><td>{total_out_abm_fet_blm}</td><td>{per_out_abm_fet_blm} %</td><td>{total_out_blm_fet_blm}</td><td>{per_out_blm_fet_blm} %</td>"
                      
                    html+="</table></center>"
                    
                    display(HTML(html))
                          
    elif((str(check_type_feature) not in type and df_feature.nunique()<=uniq_cases) and (str(check_type_outcome) not in type) and df_outcome.nunique()<=uniq_cases):
        #UNIQUE STRING FEATURE VS UNIQUE STRING OUTCOME
        if(df_feature.nunique()<=uniq_cases and df_outcome.nunique()<=uniq_cases ): #Checks if feature has less unique values
            
            uniq_sorted_ft=sorted(df_feature.unique()) #Sorts features unique values
            
            #Checks if outcome has less unique values
            uniq_sorted_ot=sorted(df_outcome.unique()) #list to put unique values for outcome
                
            #Unique occurence of features
            feature_count=np.array(list(df[feature].value_counts().sort_index()))
                
                
            #Each Unique Occurence of features and outcomes simultanously
            outcome_count = np.array(list(df.groupby([outcome,feature]).size().unstack(fill_value=0).stack()))
                
            #Percentage of each occurence of Feature
            feature_percent=[]
            for i in range(0,len(feature_count)):
                feature_percent.append(round(((feature_count[i]/total_data)*100),2))
            feature_percent=np.array(feature_percent)    
                
                
            #Percentage of Occurence of features with outcome
                
            #partioning feature with respect to outcome
            partition=int(((len(outcome_count)/len(feature_count))or(len(feature_count)/len(outcome_count))))
            #so that percent will be float
            outcome_count_float=[]
            for i in range(0,len(outcome_count)):
                outcome_count_float.append(float(outcome_count[i]))
            #Calculating outcome percentage
            outcome_percent=[]
            for i in range(0, len(outcome_count_float), len(feature_count)):
                outcome_percent.append(outcome_count_float[i:i+len(feature_count)])
                
            for i in range(0,len(feature_count)):
                for j in range(0,partition):
                    outcome_percent[j][i]=((outcome_percent[j][i]/feature_count[i])*100)
                    outcome_percent[j][i]=round(outcome_percent[j][i],2)
            outcome_percent=np.array(outcome_percent)
                
            #indexes of outcome data
            indexes=[]
            for uniq in uniq_sorted_ot:
                indexes.append(uniq_sorted_ot.index(uniq))
                
            #Display
            html+=f"Total Features {len(df)}<br>"
            html+="<center><table cellpadding=15>"
            if design==0:
                html+="<tr>"
                html+=f"<td><center>{feature}</center> </td> <td>Total No. <br>({feature})</td> <td> Percentage <br>({feature}) </td>"
                for i in indexes:
                    html+=f"<td>Total No.<br>({uniq_sorted_ot[i]})</td> <td>Percentage <br>({uniq_sorted_ot[i]})</td> "
                html+="</tr>"
                design=1
            for i in range (0,len(feature_count)):
                html+="<tr>"
                for j in range(0,partition):
                    if design_feature==0:
                        html+=f"<td></center>{uniq_sorted_ft[i]}</center></td><td>{feature_count[i]}</td><td>{feature_percent[i]}%</td>"
                    if design_feature==0:
                        html+=f"<td>{outcome_count[i]}</td><td>{outcome_percent[j][i]}%</td>"
                    else:
                        html+=f"<td>{outcome_count[i+len(feature_count)*j]}</td><td>{outcome_percent[j][i]}%</td>"
                    design_feature=1
                html+=f"</tr>"
                
                design_feature=0
            html+=f"</table></center>"
            display(HTML(html))
        #MULTIPLE NUMERICAL FEATURES VS UNIQUE STRING OUTCOME
        if (df_feature.nunique()>uniq_cases and str(check_type_feature)  in type and df_outcome.nunique()<=uniq_cases):
            mean_value=round(df_feature.mean(),2)
            uniq_sorted_ot=sorted(df_outcome.unique()) #Sorts features unique values
            #Dict for Total and percent
            total={}
            
            percent={}
            total_abm=0
            total_blm=0
            #Dict for total above and below mean and thier percentage
            total_feature_abm={}
            total_feature_blm={}
            percent_feature_abm={}
            percent_feature_blm={}
            #initializing dictionaries with 0
            uniq_sorted_ot=sorted(df_outcome.unique())
            for i in uniq_sorted_ot:
                total_feature_abm[i]=0
                total_feature_blm[i]=0
                total[i]=0
                percent[i]=0
                percent_feature_abm[i]=0
                percent_feature_blm[i]=0
            #Count below mean and above mean
            for i in range(0,len(df)):
                if(df_feature[i]>=mean_value):
                    total_abm+=1
                else:
                    total_blm+=1
            #Count for total above and below mean
            for uniq in uniq_sorted_ot:
                for i in range(0,len(df)):
                    if(df_feature[i]>=mean_value):
                        if(df[outcome][i]==uniq):
                            total_feature_abm[uniq]+=1
                    else:
                        if(df[outcome][i]==uniq):
                            total_feature_blm[uniq]+=1
            #Count total in each category
            for uniq in uniq_sorted_ot:
                total[uniq]=total_feature_abm[uniq]+total_feature_blm[uniq]
            #Count percent in each category
            for uniq in uniq_sorted_ot:
                percent[uniq]=round(total[uniq]/total_data*100,2)
                percent_feature_abm[uniq]=round(total_feature_abm[uniq]/total[uniq]*100,2)
                percent_feature_blm[uniq]=round(total_feature_blm[uniq]/total[uniq]*100,2)
                        
            #Unique occurence of features
            outcome_count=np.array(list(df[outcome].value_counts().sort_index()))
            
            #Percentage of each occurence of Feature
            outcome_percent=[]
            for i in range(0,len(outcome_count)):
                outcome_percent.append(round(((outcome_count[i]/total_data)*100),2))
            outcome_percent=np.array(outcome_percent)
            
            #Checks if outcome has less unique values
                                             
            #Display
            if design==0:
                html+=f"Total Features {len(df)}<br>"
                html+=f"Mean of {feature} = {mean_value}<br>"
                html+="<center><table cellpadding=15>"
                html+="<tr>"
                html+=f"<td><center>{feature}</center> </td> <td>Total No. <br>({feature})</td> <td> Percentage <br>({feature}) </td>"
                for uniq in uniq_sorted_ot:
                    html+=f"<td>Total Outcome<br>({uniq})</td><td>Percentage Outcome<br>({uniq})"
                html+="</td>"
                html+="</tr>"
                design=1
                
                design_feature=0
                html+=f"<tr><td>Greater Than Mean</td><td>{total_abm}</td><td>{round((total_abm/total_data)*100,2)}</td>"
                for uniq in uniq_sorted_ot:
                    html+=f"<td>{total_feature_abm[uniq]}</td><td>{percent_feature_abm[uniq]} %</td>"
                html+=f"<tr><td>Less Than Mean</td><td>{total_blm}</td><td>{round((total_blm/total_data)*100,2)}</td>"
                for uniq in uniq_sorted_ot:
                    html+=f"<td>{total_feature_blm[uniq]}</td><td>{percent_feature_blm[uniq]} %</td>"  
                html+="</table></center>"
                
                display(HTML(html))
        #UNIQUE STRING FEATURES AND MULTIPLE NUMERICAL OUTCOMES
        if(df_feature.nunique()<=uniq_cases and df_outcome.nunique()>uniq_cases and str(check_type_outcome)  in type):
            mean_value=round(df_outcome.mean(),2)
            
            #Dict for Total and percent
            total={}
            percent={}
            #Dict for total above and below mean and thier percentage
            total_outcome_abm={}
            total_outcome_blm={}
            percent_outcome_abm={}
            percent_outcome_blm={}
            #initializing dictionaries with 0
            for i in uniq_sorted_ft:
                total_outcome_abm[i]=0
                total_outcome_blm[i]=0
                total[i]=0
                percent[i]=0
                percent_outcome_abm[i]=0
                percent_outcome_blm[i]=0
            #Count for total above and below mean
            for uniq in uniq_sorted_ft:
                for i in range(0,len(df)):
                    if(df_outcome[i]>=mean_value):
                        if(df[feature][i]==uniq):
                            total_outcome_abm[uniq]+=1
                    else:
                        if(df[feature][i]==uniq):
                            total_outcome_blm[uniq]+=1
            #Count total in each category
            for uniq in uniq_sorted_ft:
                total[uniq]=total_outcome_abm[uniq]+total_outcome_blm[uniq]
            #Count percent in each category
            for uniq in uniq_sorted_ft:
                percent[uniq]=round(total[uniq]/total_data*100,2)
                percent_outcome_abm[uniq]=round(total_outcome_abm[uniq]/total[uniq]*100,2)
                percent_outcome_blm[uniq]=round(total_outcome_blm[uniq]/total[uniq]*100,2)
                                             
            #Display
            if design==0:
                html+=f"Total Features {len(df)}<br>"
                html+=f"Mean of {outcome} = {mean_value}<br>"
                html+="<center><table cellpadding=15>"
                html+="<tr>"
                html+=f"<td><center>{feature}</center> </td> <td>Total No. <br>({feature})</td> <td> Percentage <br>({feature}) </td>"
                html+=f"<td>Total No.<br>(Greater Than Mean)</td> <td>Percentage<br>(Greater Than Mean)</td><td>Total No.<br>(Less Than Mean)</td> <td>Percentage <br>(Less Than Mean)</td> "
                html+="</tr>"
                design=1
                
            design_feature=0
            for uniq in uniq_sorted_ft:
                html+="<tr>"    
                html+=f"<td>{uniq}</td> <td>{total[uniq]}</td><td>{percent[uniq]} %</td><td>{total_outcome_abm[uniq]}</td><td>{percent_outcome_abm[uniq]} %</td><td>{total_outcome_blm[uniq]}</td><td>{percent_outcome_blm[uniq]} %</td>"           
                html+=f"</tr>"
            html+="</table></center>"
            display(HTML(html))
        
    if(df_feature.nunique()>len(df)/1.5 and str(check_type_feature not in type)):
        html_alert=f"<h2><center>CAN'T_EVALUATE_ERROR:<font color='red'> {feature}</font> as <font color='red'> Feature </font> is of type <font color='red'>{check_type_feature}</font> having a large number of unique values.</center></h2>"
        display(HTML(html_alert))          
    if(df_outcome.nunique()>len(df)/1.5 and str(check_type_outcome not in type)):
        html_alert=f"<h2><center>CAN'T_EVALUATE_ERROR:<font color='red'> {outcome}</font> as <font color ='red'>Outcome </font> is of type <font color='red'>{check_type_outcome}</font> having a large number of unique values.</center></h2>"
        display(HTML(html_alert))  

<h1> UNIQUE NUMERICAL VS UNIQUE NUMERICAL </h1>
<h3>Comparing Between Pclass(unique_value_count = 3)[ 1 2 3] and Survived (unique_value_count=2)[0 1] <br>(Both Unique Numerical Values)</h3>
<h4>So choose max_unique_value_count =3 </h4>

In [17]:
Detailed_Details(df,'Pclass','Survived',3)
Detailed_Details(df,'Survived','Pclass',3)

Pclass,Total No. (Pclass),Percentage (Pclass),Total Outcome(0),Percentage Outcome(0),Total Outcome(1),Percentage Outcome(1)
1,107,25.6%,57,53.27%,50,46.73%
2,93,22.25%,63,67.74%,30,32.26%
3,218,52.15%,146,66.97%,72,33.03%


Survived,Total No. (Survived),Percentage (Survived),Total Outcome(1),Percentage Outcome(1),Total Outcome(2),Percentage Outcome(2),Total Outcome(3),Percentage Outcome(3)
0,266,63.64%,57,21.43%,63,23.68%,146,54.89%
1,152,36.36%,50,32.89%,30,19.74%,72,47.37%


<h1> MULTIPLE NUMERICAL VS UNIQUE NUMERICAL</h1>
<h3>Comparing Between Pclass(unique_value_count=3)[1 2 3] with Fare(has high unique_value_count) <br>(Pclass Has Distinct Value but Fare doesnot have.)</h3>
<h4>We set max_unique_value_count = 3 </h4>

In [18]:
Detailed_Details(df,'Pclass','Fare',3)
Detailed_Details(df,'Fare','Pclass',3)

Pclass,Total No. (Pclass),Percentage (Pclass),Total No.(Greater Than Mean),Percentage(Greater Than Mean),Total No.(Less Than Mean),Percentage (Less Than Mean)
1,107,25.6 %,77,71.96 %,30,28.04 %
2,93,22.25 %,12,12.9 %,81,87.1 %
3,218,52.15 %,8,3.67 %,210,96.33 %


<h1> MULTIPLE NUMERICAL VS MULTIPLE NUMERICAL </h1>
<h3>Comparing Between Age(has high uniqe_value_count) with Fare(has high uniqe_value_count) 
<h4>We set max_unique_value_count to 0 (doesnot really matter here) </h4>

In [19]:
Detailed_Details(df,'Age','Fare',0)
Detailed_Details(df,'Fare','Age',0)

Age,Total No. (Age),Percentage (Age),Total Outcome Greater Than Mean(35.63),Percentage Outcome Greater Than Mean(35.63),Total Outcome Less Than Mean(35.63),Percentage Outcome Less Than Mean(35.63)
Greater Than Mean(30.27),218,52.15 %,61,27.98 %,157,72.02 %
Less Than Mean(30.27),200,47.85 %,36,18.0 %,164,82.0 %


Fare,Total No. (Fare),Percentage (Fare),Total Outcome Greater Than Mean(30.27),Percentage Outcome Greater Than Mean(30.27),Total Outcome Less Than Mean(30.27),Percentage Outcome Less Than Mean(30.27)
Greater Than Mean(35.63),97,23.21 %,61,62.89 %,36,37.11 %
Less Than Mean(35.63),321,76.79 %,157,48.91 %,164,51.09 %


<h1> UNIQUE NUMERICAL VS UNIQUE STRING </h1>
<h3>Comparing Between Pclass(uniqe_value_count=3)[1 2 3] with Sex(unique_value_count=2)[female male] <br> (Both Have low unique_value_counts)</h3>
<h4>We set max_unique_value_count=3</h4>

In [20]:
Detailed_Details(df,'Pclass','Sex',3)
Detailed_Details(df,'Sex','Pclass',3)


Pclass,Total No. (Pclass),Percentage (Pclass),Total Outcome(female),Percentage Outcome(female),Total Outcome(male),Percentage Outcome(male)
1,107,25.6%,50,46.73%,57,53.27%
2,93,22.25%,30,32.26%,63,67.74%
3,218,52.15%,72,33.03%,146,66.97%


Sex,Total No. (Sex),Percentage (Sex),Total Outcome(1),Percentage Outcome(1),Total Outcome(2),Percentage Outcome(2),Total Outcome(3),Percentage Outcome(3)
female,152,36.36%,50,32.89%,30,19.74%,72,47.37%
male,266,63.64%,57,21.43%,63,23.68%,146,54.89%



<h1> MULTIPLE NUMERICAL VS UNIQUE STRING </h1>
<h3>Comparing Between Fare(has high unique_value_count) with Embarked(uniqe_value_count=3)[C Q S] </h3>
<h4>We set max_uniqe_value_count=3</h4>

In [21]:

Detailed_Details(df,'Fare','Embarked',3)
Detailed_Details(df,'Embarked','Fare',3)

Embarked,Total No. (Embarked),Percentage (Embarked),Total No.(Greater Than Mean),Percentage(Greater Than Mean),Total No.(Less Than Mean),Percentage (Less Than Mean)
C,102,24.4 %,45,44.12 %,57,55.88 %
Q,46,11.0 %,1,2.17 %,45,97.83 %
S,270,64.59 %,51,18.89 %,219,81.11 %


<h1> UNIQUE STRING VS UNIQUE STRING </h1>
<h3>Comparing Between Sex(uniqe_value_count=2)[female male] with Embarked(uniqe_value_count=3)[C Q S] <br> (Both have low unique_value_counts and are of string type)</h3>
<h4>We set max_uniqe_value_count=3</h4>

In [22]:
Detailed_Details(df,'Sex','Embarked',3)
Detailed_Details(df,'Embarked','Sex',3)

Sex,Total No. (Sex),Percentage (Sex),Total No.(C),Percentage (C),Total No.(Q),Percentage (Q),Total No.(S),Percentage (S)
female,152,36.36%,40,26.32%,24,15.79%,88,57.89%
male,266,63.64%,62,23.31%,22,8.27%,182,68.42%


Embarked,Total No. (Embarked),Percentage (Embarked),Total No.(female),Percentage (female),Total No.(male),Percentage (male)
C,102,24.4%,40,39.22%,62,60.78%
Q,46,11.0%,24,52.17%,22,47.83%
S,270,64.59%,88,32.59%,182,67.41%


<h3>Trying with String having high number of uniqe_value_count</h3>
<h4>Like Name </h4>
<h4>Here max_uniqe_value_count doesnot matter much</h4>

In [23]:
Detailed_Details(df,'Pclass','Name',3)

Detailed_Details(df,'Name','Pclass',3)